# Espectrograma para la Detección de Aves: Documentación del Proyecto

## Documentación

El archivo **`Bird_Recognition_Prosperina_Generate_Spectograms`** se enmarca en un proyecto de investigación académica centrado en la detección de aves a partir de sus vocalizaciones. El objetivo principal de este archivo es la generación de espectrogramas o imágenes de los audios de aves, los cuales se encuentran segmentados en tramos de 2 segundos. Estos datos se ubican en la carpeta **`Bird-Audio-Classification-Prosperina\Dataset-Birds\training`**.

## Herramientas y Bibliotecas Utilizadas

- **`Python`**: Lenguaje de programación principal utilizado en el proyecto.
- **`Librerías de Procesamiento de Señales de Audio`**:
  - **`Librosa`**: Utilizada para cargar y visualizar espectrogramas de audio.
  - **`Noisereduce`**: Empleada para reducir el ruido de las grabaciones de audio.
- **`Librerías de Aprendizaje Automático`**:
  - **`TensorFlow y Keras`**: Utilizadas para la implementación de modelos de redes neuronales.
- **`Otras Bibliotecas`**:
  - **`Pandas`**: Para el manejo de datos tabulares.
  - **`Matplotlib`**: Utilizada para la visualización de datos y gráficos.
  - **`PIL (Python Imaging Library)`**: Empleada para el procesamiento de imágenes.

In [1]:
## Configuración Inicial
import os
import pandas as pd
import matplotlib.pyplot as plt
import librosa.display
import noisereduce as nr
import numpy as np
import datetime
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import tensorflow as tf 
import keras
from keras import layers
from keras.applications import EfficientNetB1
from PIL import Image
from sklearn.calibration import LabelEncoder
import hashlib

c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Preparación del Conjunto de Datos

In [2]:
# Ruta a la carpeta con los archivos de audio
folder_path = "./Dataset-Integradora-Bird/training/"

# Obtener la lista de archivos en la carpeta
file_list = os.listdir(folder_path)
# Filtrar la lista para obtener solo los archivos de audio
audio_files = [file for file in file_list if file.endswith('.wav')]

# Crear una lista de diccionarios con etiqueta y ruta
data = []
for audio_file in audio_files:
    label, _ = audio_file.split('.', 1)
    data.append({'label': label, 'path': os.path.join(folder_path, audio_file)})

# Crear el DataFrame
df = pd.DataFrame(data)

class_scientificnames = df['label'].unique()
print(class_scientificnames)
print(df['label'].value_counts())


['Amazona Autamnails' 'Brotogeris pyrrhoptera' 'Campephilus gayaquilensis'
 'Crypturellus tansfasciatus' 'Cyanocorax mystacalis' 'Euphonia saturata'
 'Lathrotriccus griseipectus' 'Ortalis erythroptera'
 'Pachyramphus spodiurus' 'Picumnus sclateri' 'Pseudastur occidentalis'
 'Psittacara erythrogenys' 'Trogon mesurus']
label
Amazona Autamnails            650
Psittacara erythrogenys       522
Ortalis erythroptera          402
Cyanocorax mystacalis         313
Brotogeris pyrrhoptera        253
Lathrotriccus griseipectus    244
Campephilus gayaquilensis     187
Crypturellus tansfasciatus    165
Trogon mesurus                142
Pseudastur occidentalis        96
Picumnus sclateri              64
Euphonia saturata              52
Pachyramphus spodiurus         34
Name: count, dtype: int64


## Generación de Espectrogramas Mel y Creación de Imágenes
La función `createSpectogramImageMel` se encarga de generar y guardar un espectrograma Mel a partir de una señal de audio dada. A continuación se presenta una explicación más detallada de cada paso dentro de esta función:

1. **Configuración de parámetros de la transformada de Fourier de tiempo corto (STFT):**
   - `n_fft`: Este parámetro define el tamaño de la ventana utilizada en la transformada de Fourier de tiempo corto (STFT). Especifica el número de muestras en cada ventana donde se realizará la STFT. En este caso, se utiliza una potencia de 2 para mejorar el rendimiento computacional.
   - `hop_length`: Este parámetro determina la cantidad de muestras que se moverá la ventana en cada desplazamiento. Es decir, la cantidad de traslape entre ventanas adyacentes. Aquí, se establece en un cuarto de la longitud de la ventana (`n_fft`) para obtener un buen equilibrio entre resolución temporal y frecuencial.

2. **Cálculo del espectrograma de magnitud:**
   - Se utiliza la función `librosa.stft` para calcular la STFT de la señal de audio con los parámetros especificados. Luego, se calcula el valor absoluto de la STFT para obtener el espectrograma de magnitud.

3. **Configuración de la ruta de guardado del espectrograma:**
   - Se genera un nombre de archivo único para el espectrograma basado en la etiqueta de la clase y el hash de la señal de audio. Esto asegura que cada espectrograma tenga un nombre único.
   - La ruta de guardado se configura combinando el directorio de guardado (`save_dir`) y el nombre de archivo generado.

4. **Visualización y configuración del espectrograma:**
   - Se crea una figura y un eje utilizando `plt.subplots()` para visualizar el espectrograma.
   - Se normaliza el espectrograma utilizando la función `librosa.amplitude_to_db()` para convertir la magnitud de la STFT en decibelios (dB).
   - Se utiliza `librosa.display.specshow()` para mostrar el espectrograma Mel en la figura.

5. **Ajuste y eliminación de elementos no deseados:**
   - Se eliminan títulos, etiquetas de ejes y barras de color para obtener una imagen limpia del espectrograma.
   - Se ajusta el tamaño de la figura y se elimina el espacio en blanco alrededor de la imagen.

6. **Guardado del espectrograma como archivo de imagen:**
   - Se utiliza `plt.savefig()` para guardar la figura como un archivo de imagen en formato JPEG.
   - La función crea el directorio de guardado si no existe y guarda la imagen en la ruta especificada.

7. **Retorno de la ruta de la imagen guardada:**
   - Se retorna la ruta completa del archivo de imagen recién guardado.

In [4]:
def createSpectogramImageMel(signal, sr, label, save_dir="Dataset-Images-Mel", img_size=(224, 224)):
    n_fft = 2**11  # Nro. de muestras para cada ventana donde se va a realizar la stft
    hop_length = int(n_fft/4)  # Cuántas muestras se va a mover la ventana en cada desplazamiento (hop: salto)
    stft = librosa.stft(signal, n_fft=n_fft, hop_length=hop_length, win_length=None, window='hann')
    spectrogram_magnitude = np.abs(stft)
    
    # Configurar la ruta de guardado
    filename = f"{label}_{hashlib.sha1(signal.tobytes()).hexdigest()[:8]}_{len(os.listdir(save_dir))}.jpeg"
    save_path = os.path.join(save_dir, filename)

    # Visualizar el espectrograma normalizado sin valores de frecuencia
    fig, ax = plt.subplots(1, 1, figsize=(img_size[0]/100, img_size[1]/100))  # Ajustar el tamaño de la figura
    fig.patch.set_facecolor('white')
    spectrogram_dB = librosa.amplitude_to_db(spectrogram_magnitude, ref=np.max)
    librosa.display.specshow(spectrogram_dB, y_axis='mel', fmin=0, fmax=sr/2, x_axis='time', sr=sr, hop_length=hop_length)

    # Eliminar títulos y etiquetas
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')

    plt.colorbar(format='%+2.0f dB').remove()

    # Eliminar etiquetas de ejes x e y
    ax.axis('off')

    # Ajustar el tamaño de la imagen al guardarla
    fig.tight_layout(pad=0)

    # Guardar la figura en el archivo de imagen con el tamaño especificado
    os.makedirs(save_dir, exist_ok=True)
    plt.savefig(save_path, bbox_inches="tight", pad_inches=0, dpi=100)
    plt.close()

    return save_path

## Pruebas con Datos Aleatorios y Visualización de Resultados

In [ ]:
# Seleccionar una muestra aleatoria del conjunto de datos
import IPython

random_data = df.sample()
random_path = random_data['path'].values[0]  # Acceder al valor de la primera fila
random_label = random_data['label'].values[0]  # Acceder al valor de la primera fila

# Mostrar la ruta aleatoria seleccionada y la etiqueta correspondiente
print("Ruta aleatoria:", random_path)
print("Etiqueta aleatoria:", random_label)

# Cargar la señal de audio desde la ruta aleatoria seleccionada
signal, sr = librosa.load(random_path, sr=None)  # sr = sampling rate

# Visualizar la señal de audio como un archivo de audio interactivo
IPython.display.Audio(signal, rate=sr)

# Generar el espectrograma Mel correspondiente a la señal de audio y guardar la imagen
image_path = createSpectogramImageMel(signal=signal, sr=sr, label=random_label)

# Imprimir la ruta de la imagen guardada
print("Ruta de la imagen del espectrograma:", image_path)


## Generación de Datos de Entrenamiento Adicionales
1. **Inicialización de `data_training`:**
   - La variable `data_training` se inicializa como una lista vacía. Este será el contenedor donde almacenaremos los datos de entrenamiento adicionales que generaremos.

2. **Iteración sobre el DataFrame original (`df`):**
   - Utilizamos un bucle `for` junto con `df.iterrows()` para iterar sobre cada fila del DataFrame original `df`. Cada fila representa un archivo de audio y su correspondiente etiqueta.

3. **Procesamiento de cada muestra de audio:**
   - Para cada fila del DataFrame, extraemos la ruta del archivo de audio (`path`) y su etiqueta (`label`).
   - Cargamos la señal de audio utilizando la función `librosa.load()`, especificando `sr=None` para mantener la frecuencia de muestreo original.
   - Aplicamos la reducción de ruido a la señal de audio utilizando `nr.reduce_noise()` de la librería `noisereduce`.
   
4. **Generación de datos adicionales para ciertas clases:**
   - Verificamos si la etiqueta no está en una lista específica de nombres de clases. Si no está en esa lista, generamos variaciones de la señal de audio y creamos espectrogramas Mel para cada variación.
   - Las variaciones incluyen desplazamientos de tono (`y_roll` y `y_pitch2`) y una reducción de la amplitud (`y_scaled`).
   - Para cada variación, creamos un espectrograma Mel correspondiente utilizando la función `createSpectogramImageMel()` y guardamos la ruta del archivo de imagen junto con su etiqueta en la lista `data_training`.
   
5. **Procesamiento adicional para clases específicas:**
   - Si la etiqueta está en una lista específica de nombres de clases, generamos una variación adicional de la señal de audio (`y_pitch3`) y creamos un espectrograma Mel correspondiente.
   
6. **Creación del DataFrame `datframe`:**
   - Una vez que hemos generado todas las muestras de entrenamiento adicionales, creamos un nuevo DataFrame llamado `datframe` a partir de la lista `data_training`.

7. **Impresión de la distribución de etiquetas:**
   - Finalmente, imprimimos la distribución de las etiquetas en el conjunto de datos de entrenamiento utilizando `datframe[1].value_counts()`.


In [26]:
data_training = []

# Iterar sobre cada fila del DataFrame original
for index, row in df.iterrows():
    path = row["path"]
    label = row["label"]
    signal, sr = librosa.load(path, sr=None)  # sr = sampling rate
    signal = nr.reduce_noise(y=signal, y_noise=signal, prop_decrease=1, sr=sr)

    # Generar datos adicionales solo para ciertas clases
    if label not in ["Amazona Autamnails", "Psittacara erythrogenys", "Ortalis erythroptera", "Cyanocorax mystacalis", "Brotogeris pyrrhoptera", "Lathrotriccus griseipectus"]:
        # Manipulación de la señal de audio para generar variaciones
        y_roll = np.roll(signal, 7000)
        y_pitch2 = np.roll(signal, 30000)
        y_scaled = 0.6 * signal  # Reducir la amplitud en un 40%

        # Crear espectrogramas Mel para las variaciones de la señal
        pitchspectogram2_path = createSpectogramImageMel(signal=y_pitch2, sr=sr, label=label)
        pitchspectogram_path = createSpectogramImageMel(signal=y_roll, sr=sr, label=label)
        scaledspectogram_path = createSpectogramImageMel(signal=y_scaled, sr=sr, label=label)
        spectogram_path = createSpectogramImageMel(signal=signal, sr=sr, label=label)

        # Agregar las rutas de los espectrogramas y las etiquetas al conjunto de datos
        data_training.append([pitchspectogram2_path, label])
        data_training.append([spectogram_path, label])
        data_training.append([pitchspectogram_path, label])
        data_training.append([scaledspectogram_path, label])

    else:
        # Generar espectrogramas Mel para las señales de audio sin manipulación adicional
        spectogram_path = createSpectogramImageMel(signal=signal, sr=sr, label=label)
        data_training.append([spectogram_path, label])

    # Generar datos adicionales solo para ciertas clases
    if label in ["Pseudastur occidentalis","Picumnus sclateri","Euphonia saturata","Pachyramphus spodiurus"]:
        y_pitch3 = np.roll(signal, 80000)  
        pitchspectogram_path = createSpectogramImageMel(signal=y_pitch3, sr=sr, label=label)
        data_training.append([pitchspectogram_path, label])

# Crear un DataFrame con los datos de entrenamiento adicionales generados
datframe = pd.DataFrame(data_training)

# Imprimir la distribución de las etiquetas en el conjunto de datos de entrenamiento
print(datframe[1].value_counts())